In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
from tqdm import tqdm


In [2]:
# df = pd.read_csv('data/ner_dataset.csv', encoding='latin-1')
# df = df.rename(columns={'Sentence #': 'sentID'})
# df['sentID'].fillna(method='ffill', inplace=True)
# df.head()

In [47]:
CONLL_dir = 'data/NER-2021/NER/NER-Data-Muc-CONLL'
all_data = []
wrong_fn = []
sentence_id = 0

for fn in glob(f'{CONLL_dir}/*.conll'):
    f = open(fn, 'r')
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line:
            data = line.split()
            try:
                all_data.append({
                    'sentID': sentence_id,
                    'Word': data[0],
                    'Tag': data[3],
                    'filename': fn.split('/')[-1],
                    'covid_data': False
                })
            except:
                wrong_fn.append(fn)
        else:
            sentence_id += 1
            

print(len(all_data))
print(set(wrong_fn))

257247
{'data/NER-2021/NER/NER-Data-Muc-CONLL/0_dev_thethao_0006.conll', 'data/NER-2021/NER/NER-Data-Muc-CONLL/doisong_0005.conll', 'data/NER-2021/NER/NER-Data-Muc-CONLL/giaitri_0020.conll', 'data/NER-2021/NER/NER-Data-Muc-CONLL/thegioi_0075.conll'}


In [48]:
df = pd.DataFrame(all_data)
df.head()

,sentID,Word,Tag,filename,covid_data
0,0,Cội,O,thegioi_0029.conll,False
1,0,nguồn,O,thegioi_0029.conll,False
2,0,thảm,O,thegioi_0029.conll,False
3,0,kịch,O,thegioi_0029.conll,False
4,0,Rohingya,O,thegioi_0029.conll,False


In [49]:
print(df['Tag'].unique())

['O' 'B-LOCATION' 'B-ORGANIZATION' 'I-ORGANIZATION' 'I-LOCATION'
 'B-PERSONTYPE' 'I-PERSONTYPE' 'B-DATETIME' 'B-MISCELLANEOUS'
 'I-MISCELLANEOUS' 'B-QUANTITY' 'I-QUANTITY' 'B-EVENT' 'I-EVENT'
 'B-PERSON' 'I-PERSON' 'B-LOCATION-GPE' 'I-LOCATION-GPE' 'B-DATETIME-DATE'
 'I-DATETIME-DATE' 'B-DATETIME-DURATION' 'I-DATETIME-DURATION'
 'B-QUANTITY-NUM' 'B-DATETIME-TIME' 'I-DATETIME-TIME' 'B-PRODUCT'
 'I-PRODUCT' 'B-QUANTITY-TEM' 'I-QUANTITY-TEM' 'B-QUANTITY-ORD'
 'I-QUANTITY-ORD' 'B-PRODUCT-COM' 'B-EVENT-SPORT' 'I-EVENT-SPORT'
 'B-QUANTITY-DIM' 'I-DATETIME' 'B-ORGANIZATION-SPORTS'
 'I-ORGANIZATION-SPORTS' 'B-LOCATION-STRUC' 'I-LOCATION-STRUC'
 'I-QUANTITY-NUM' 'B-QUANTITY-AGE' 'I-QUANTITY-AGE' 'B-*' 'I-*'
 'B-QUANTITY-PER' 'I-QUANTITY-PER' 'B-DATETIME-TIMERANGE'
 'I-DATETIME-TIMERANGE' 'B-EVENT-GAMESHOW' 'I-EVENT-GAMESHOW' 'B-URL'
 'B-LOCATION-GEO' 'I-LOCATION-GEO' 'I-QUANTITY-DIM' 'B-EVENT-NATURAL'
 'I-EVENT-NATURAL' 'B-QUANTITY-CUR' 'I-QUANTITY-CUR' 'B-SKILL' 'I-SKILL'
 'B-DATETIME-SET' 'I-

In [50]:
df.to_csv('data/NER_VLSP_2021.csv')

## Vin Covid dataset

In [51]:
CONLL_dir = 'data/PhoNER_COVID19/data/word'
all_data = []
wrong_fn = []
sentence_id = 0

for fn in glob(f'{CONLL_dir}/*.conll'):
    f = open(fn, 'r')
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line:
            data = line.split()
            try:
                all_data.append({
                    'sentID': sentence_id,
                    'Word': data[0],
                    'Tag': data[1],
                    'filename': fn.split('/')[-1],
                    'covid_data': True
                })
            except:
                wrong_fn.append(fn)
        else:
            sentence_id += 1
            
    print(len(all_data))
    print(set(wrong_fn))
    df = pd.DataFrame(all_data)
    csv_name = 'data/covid_{}.csv'.format(os.path.basename(fn).split('.')[0])
    print(csv_name)
    df.to_csv(csv_name)    

85678
set()
data/covid_test_word.csv
218189
set()
data/covid_train_word.csv
274472
set()
data/covid_dev_word.csv


## Merge VLSP 2021 dataset and Vin Covid Dataset

In [53]:
vlsp_train = pd.read_csv('data/NER_VLSP_2021.csv', index_col=0)
vin_train = pd.read_csv('data/covid_train_word.csv', index_col=0)
merge_train_df = pd.merge(vlsp_train, vin_train, how='outer')
merge_train_df.to_csv('data/merge_vlsp_CovidVin.csv')

In [46]:
merge_train_df.head()

,sentID,Word,Tag,filename,covid_data
0,0,Từ,O,test_word.conll,True
1,0,24,B-DATE,test_word.conll,True
2,0,-,I-DATE,test_word.conll,True
3,0,-,I-DATE,test_word.conll,True
4,0,-,I-DATE,test_word.conll,True


## Preprocess Vin Covida dataset

In [11]:
import pandas as pd
from glob import glob
import os

df = pd.read_csv('data/covid_train_word.csv', index_col=0)
df.head(50)



CONLL_dir = 'data/PhoNER_COVID19/data/word'
all_data = []
wrong_fn = []
sentence_id = 0

for fn in glob(f'{CONLL_dir}/*.conll'):
    f = open(fn, 'r')
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line:
            data = line.split()
            try:
                word = data[0]
                tag = data[1]
                if '_' not in word:
                    all_data.append({
                        'sentID': sentence_id,
                        'Word': word,
                        'Tag': tag,
                        'filename': fn.split('/')[-1],
                        'covid_data': True
                    })
                else:
                    words = word.split('_')
                    for i, word in enumerate(words):
                        if i > 0 and tag != 'O':
                            all_data.append({
                                'sentID': sentence_id,
                                'Word': word,
                                'Tag': 'I-' + tag[2:],
                                'filename': fn.split('/')[-1],
                                'covid_data': True
                            })
                        else:
                            all_data.append({
                                'sentID': sentence_id,
                                'Word': word,
                                'Tag': tag,
                                'filename': fn.split('/')[-1],
                                'covid_data': True
                            })
            except:
                wrong_fn.append(fn)
        else:
            sentence_id += 1
            
    print(len(all_data))
    print(set(wrong_fn))
    df = pd.DataFrame(all_data)
    csv_name = 'data/covid_{}.csv'.format(os.path.basename(fn).split('.')[0])
    print(csv_name)
    df.to_csv(csv_name)    

108354
set()
data/covid_test_word.csv
275895
set()
data/covid_train_word.csv
347221
set()
data/covid_dev_word.csv
